<h1 style = "font-size:3rem;color:orange">Examen 2022_1</h1>

Utilizar el conjunto de datos "dataset_exam.npy" para resolver el ejercicio. Tener en cuenta que la última columna corresponde a la clase.

In [ ]:
# Librerías
